In [88]:
import pandas as pd
import numpy as np

In [96]:
df = pd.read_csv("Z:/00_ETL/Collaborative_filtering/data_4.txt", sep = "\t")
df.columns = ['UserID','MovieID','content_type','content_length_seconds_total','pause_count',
              'seconds_viewed','seconds_paused','offer_group_desc']
              

In [97]:
movies_df = pd.read_csv("../Collaborative_filtering/data/movies_df.txt", sep = '\t', header = None, encoding = 'ISO-8859-1')
movies_df.columns = ['Title','MovieID']
movies_df.head()

,Title,MovieID
0,Switch - Ein mörderischer Tausch,B00P5GGIQY
1,Varsity Blues,B00I8QYZDQ
2,Holy Flying Circus,B01B1M5HRG
3,Tuscan Wedding - Hochzeit auf Italienisch [dt....,B01E85784S
4,Just Married [OV],B00KDW2RSI


In [98]:
print(df.shape)
df['share_viewed'] = df['seconds_viewed'] / df['content_length_seconds_total']
df.head()

(6005798, 8)


,UserID,MovieID,content_type,content_length_seconds_total,pause_count,seconds_viewed,seconds_paused,offer_group_desc,share_viewed
0,A13GSUGTAOIT81,B00FZITHLK,Movie EST,14880,9,6949.248,2723.0,RENTAL,0.467019
1,A1965EQ7GVIP0Q,B01MS3SLVH,Movie EST,23760,6,6100.384,451.0,RENTAL,0.256750
2,A1XG9CQ2NVDJMP,B00ILFE8W2,Movie EST,25200,3,3925.284,12.0,RENTAL,0.155765
3,A23DRS92C2SITH,B00IK6JQSI,Movie EST,33120,1,7398.223,7.0,RENTAL,0.223376
4,A3HLHTE3XT5MDJ,B01MXTSU33,Movie EST,13920,11,9996.352,1114.0,RENTAL,0.718129


## Create ratings

In [99]:
df = df.dropna(axis = 0)

df['Rating'] = 0
df.loc[df['share_viewed'] <= 0.20, 'Rating'] = 1
df.loc[(df['share_viewed'] > 0.20) & (df['share_viewed'] <= 0.40), 'Rating'] = 2
df.loc[(df['share_viewed'] > 0.40) & (df['share_viewed'] <= 0.60), 'Rating'] = 3
df.loc[(df['share_viewed'] > 0.60) & (df['share_viewed'] <= 0.80), 'Rating'] = 4
df.loc[(df['share_viewed'] > 0.80), 'Rating'] = 5

In [100]:
df.head()

,UserID,MovieID,content_type,content_length_seconds_total,pause_count,seconds_viewed,seconds_paused,offer_group_desc,share_viewed,Rating
0,A13GSUGTAOIT81,B00FZITHLK,Movie EST,14880,9,6949.248,2723.0,RENTAL,0.467019,3
1,A1965EQ7GVIP0Q,B01MS3SLVH,Movie EST,23760,6,6100.384,451.0,RENTAL,0.256750,2
2,A1XG9CQ2NVDJMP,B00ILFE8W2,Movie EST,25200,3,3925.284,12.0,RENTAL,0.155765,1
3,A23DRS92C2SITH,B00IK6JQSI,Movie EST,33120,1,7398.223,7.0,RENTAL,0.223376,2
4,A3HLHTE3XT5MDJ,B01MXTSU33,Movie EST,13920,11,9996.352,1114.0,RENTAL,0.718129,4


In [101]:
df['Rating'].unique()

array([3, 2, 1, 4, 5], dtype=int64)

## Pivot the matrix, make subset 

In [102]:
ratings_df = df[['UserID','MovieID','Rating']].sample(frac = 0.005)

In [103]:
print(ratings_df.shape)
ratings_df.head()

(25244, 3)


,UserID,MovieID,Rating
4264845,AY36LY5BXKCQZ,B00EVA6UEK,3
4193775,A2FBXXTAH6UZWA,B00IKVKUEM,1
2064430,A1DBEF2A64AEFE,B06XKWQFTH,5
5930454,A20LXXMSWI78IL,B01MT0YEEB,1
356183,A14BUCTH9C3AOU,B072Y4S9X8,2


In [139]:
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
print(R_df.shape)
R_df.head()

(24750, 3743)


MovieID,B00EQXEB5W,B00EQXEKJ4,B00EQXEMZ6,B00EQXER4W,B00EQXFI3Q,B00EQXFXZ4,B00EQY9TRG,B00EQYTT9O,B00EQYTZZC,B00EQYU3HG,...,B078JVS3Z6,B078MY9XWW,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WFJLY8,B078WG5TT5,B078YTVBP4,B0791STYGM,B07938RRYC
UserID,,,,,,,,,,,,,,,,,,,,,
A00176501NI9ZN1E2ZXZI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0069633278HXQEC3J32C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01384843EGV1ZALLXJSN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01758403D3ACITCHEJYP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0188172QJ14Y519QK37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
#df.to_pickle(R_df)
R_df.shape

(24750, 3743)

In [201]:
R_df.head()

MovieID,B00EQXEB5W,B00EQXEKJ4,B00EQXEMZ6,B00EQXER4W,B00EQXFI3Q,B00EQXFXZ4,B00EQY9TRG,B00EQYTT9O,B00EQYTZZC,B00EQYU3HG,...,B078JVS3Z6,B078MY9XWW,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WFJLY8,B078WG5TT5,B078YTVBP4,B0791STYGM,B07938RRYC
UserID,,,,,,,,,,,,,,,,,,,,,
A00176501NI9ZN1E2ZXZI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0069633278HXQEC3J32C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01384843EGV1ZALLXJSN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01758403D3ACITCHEJYP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0188172QJ14Y519QK37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [202]:
t = R_df.reset_index()

In [205]:
t.head()

MovieID,UserID,B00EQXEB5W,B00EQXEKJ4,B00EQXEMZ6,B00EQXER4W,B00EQXFI3Q,B00EQXFXZ4,B00EQY9TRG,B00EQYTT9O,B00EQYTZZC,...,B078JVS3Z6,B078MY9XWW,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WFJLY8,B078WG5TT5,B078YTVBP4,B0791STYGM,B07938RRYC
0,A00176501NI9ZN1E2ZXZI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A0069633278HXQEC3J32C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A01384843EGV1ZALLXJSN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A01758403D3ACITCHEJYP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A0188172QJ14Y519QK37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Denorm the data, normalize by each users mean

In [106]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

## SVD

In [107]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [108]:
sigma = np.diag(sigma)

In [186]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [187]:
print(preds_df.shape)
preds_df.head()

(24750, 3743)


MovieID,B00EQXEB5W,B00EQXEKJ4,B00EQXEMZ6,B00EQXER4W,B00EQXFI3Q,B00EQXFXZ4,B00EQY9TRG,B00EQYTT9O,B00EQYTZZC,B00EQYU3HG,...,B078JVS3Z6,B078MY9XWW,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WFJLY8,B078WG5TT5,B078YTVBP4,B0791STYGM,B07938RRYC
0,0.001356,0.001356,0.001355,0.001356,0.001356,0.001356,0.001356,0.001357,0.001356,0.001356,...,0.001358,0.001356,0.001356,0.001357,0.001356,0.001356,0.001356,0.001356,0.001356,0.001356
1,0.001355,0.001355,0.001354,0.001355,0.001355,0.001355,0.001355,0.001356,0.001355,0.001355,...,0.001357,0.001355,0.001355,0.001356,0.001355,0.001355,0.001355,0.001355,0.001355,0.001355
2,0.000271,0.000271,0.000270,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,...,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271
3,0.000271,0.000271,0.000270,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,...,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271
4,0.000271,0.000271,0.000270,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,...,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271


In [213]:
new_pred = pd.concat([t['UserID'],preds_df],axis = 1)
new_pred = new_pred.set_index("UserID")
new_pred.head()

,B00EQXEB5W,B00EQXEKJ4,B00EQXEMZ6,B00EQXER4W,B00EQXFI3Q,B00EQXFXZ4,B00EQY9TRG,B00EQYTT9O,B00EQYTZZC,B00EQYU3HG,...,B078JVS3Z6,B078MY9XWW,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WFJLY8,B078WG5TT5,B078YTVBP4,B0791STYGM,B07938RRYC
UserID,,,,,,,,,,,,,,,,,,,,,
A00176501NI9ZN1E2ZXZI,0.001356,0.001356,0.001355,0.001356,0.001356,0.001356,0.001356,0.001357,0.001356,0.001356,...,0.001358,0.001356,0.001356,0.001357,0.001356,0.001356,0.001356,0.001356,0.001356,0.001356
A0069633278HXQEC3J32C,0.001355,0.001355,0.001354,0.001355,0.001355,0.001355,0.001355,0.001356,0.001355,0.001355,...,0.001357,0.001355,0.001355,0.001356,0.001355,0.001355,0.001355,0.001355,0.001355,0.001355
A01384843EGV1ZALLXJSN,0.000271,0.000271,0.000270,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,...,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271
A01758403D3ACITCHEJYP,0.000271,0.000271,0.000270,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,...,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271
A0188172QJ14Y519QK37,0.000271,0.000271,0.000270,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,...,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271,0.000271


## Making movie recommendations

In [262]:
def recommend_movies(userID, num_recommendations=3):

    sorted_user_predictions = new_pred.loc[UserID].sort_values(ascending=False)
    user_data = ratings_df[ratings_df['UserID'] == UserID]
    user_full = user_data.merge(movies_df,left_on="MovieID",
                            right_on="MovieID",how ="left").sort_values(['Rating'], ascending = False)
    
    movies_not_watched = movies_df[ ~movies_df['MovieID'].isin(user_full['MovieID'])]
    sorted_user_preds = pd.DataFrame(sorted_user_predictions.reset_index())
    sorted_user_preds.columns = ['MovieID','predictions']
    recommendation = pd.merge(left = movies_not_watched,
                              right = sorted_user_preds, left_on="MovieID",right_on="MovieID",how = 'left')
    recommendation = recommendation.sort_values(by = 'predictions', ascending = False)[0:num_recommendations]
    
    return user_full, recommendation

In [263]:
x,y = recommend_movies("A00176501NI9ZN1E2ZXZI",4)

In [264]:
x

,UserID,MovieID,Rating,Title
0,A00176501NI9ZN1E2ZXZI,B019G5VVLG,5,The Big Short [dt./OV]


In [265]:
y

,Title,MovieID,predictions
9467,Kevin - Allein in New York,B018G6YWQC,0.001534
2953,Winterkartoffelknödel,B00VU13IFM,0.001415
10623,Harry Potter und die Kammer des Schreckens,B00HX2J39W,0.001405
10384,Pitch Perfect 2 [dt./OV],B014PIFLXI,0.001396
